In [18]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
from citipy import citipy
# Import the API key.
from config import g_key

In [19]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Bredasdorp,ZA,2021-04-25 22:29:24,-34.5322,20.0403,55.40,94,100,3.44
1,1,Marabba,SD,2021-04-25 22:29:25,12.3500,32.1833,87.58,43,100,16.28
2,2,Hobart,AU,2021-04-25 22:26:01,-42.8794,147.3294,57.99,58,75,17.27
3,3,Bulaevo,KZ,2021-04-25 22:29:26,54.9056,70.4439,43.07,85,50,6.24
4,4,Atuona,PF,2021-04-25 22:29:27,-9.8000,-139.0333,79.99,73,44,15.03


In [20]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [21]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [22]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [25]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [27]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Marabba,SD,2021-04-25 22:29:25,12.3500,32.1833,87.58,43,100,16.28
4,4,Atuona,PF,2021-04-25 22:29:27,-9.8000,-139.0333,79.99,73,44,15.03
13,13,Juazeiro,BR,2021-04-25 22:25:50,-9.4117,-40.4986,84.20,61,0,5.75
17,17,Bireun,ID,2021-04-25 22:29:34,5.2030,96.7009,77.49,83,100,3.49
22,22,Butaritari,KI,2021-04-25 22:27:28,3.0707,172.7902,82.81,74,15,12.08
27,27,Salinopolis,BR,2021-04-25 22:24:44,-0.6136,-47.3561,79.50,84,54,8.28
28,28,Tual,ID,2021-04-25 22:25:12,-5.6667,132.7500,83.41,63,100,6.69
32,32,Faanui,PF,2021-04-25 22:29:39,-16.4833,-151.7500,81.03,64,27,2.95
39,39,Camapua,BR,2021-04-25 22:29:43,-19.5314,-54.0439,76.48,48,94,7.00
47,47,Rikitea,PF,2021-04-25 22:24:49,-23.1203,-134.9692,78.13,74,98,7.83


In [28]:
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [29]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Marabba,SD,87.58,12.3500,32.1833,
4,Atuona,PF,79.99,-9.8000,-139.0333,
13,Juazeiro,BR,84.20,-9.4117,-40.4986,
17,Bireun,ID,77.49,5.2030,96.7009,
22,Butaritari,KI,82.81,3.0707,172.7902,
27,Salinopolis,BR,79.50,-0.6136,-47.3561,
28,Tual,ID,83.41,-5.6667,132.7500,
32,Faanui,PF,81.03,-16.4833,-151.7500,
39,Camapua,BR,76.48,-19.5314,-54.0439,
47,Rikitea,PF,78.13,-23.1203,-134.9692,


In [30]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [31]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [17]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]